In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine 
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from skmultilearn.adapt import mlknn
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

In [3]:
if not os.path.isfile("train.db"):
    engine = create_engine('sqlite:///train.db')
    chunk =  180000
    i = 0
    indexs = 1
    for df in pd.read_csv('Train.csv', chunksize= chunk,names=['Id', 'Title', 'Body', 'Tags'],iterator=True, encoding='utf-8'):
        df.index += indexs
        i+=1
        print('{} rows'.format(i*chunk))
        df.to_sql('data', engine, if_exists='append')
        indexs = df.index[-1] + 1


In [4]:
if os.path.isfile('train.db'):
    con = sqlite3.connect('train.db')
    num_rows = pd.read_sql_query("""SELECT count(*) FROM data""", con)
    print("Number of rows : ",num_rows['count(*)'].values[0])
    con.close()

Number of rows :  6034196


In [13]:
if os.path.isfile("train.db"):
    con = sqlite3.connect("train.db")
    duplicates = pd.read_sql_query("""SELECT Title, Tags, COUNT(*) as count FROM data GROUP BY Title, Tags """, con)
    con.close()

In [14]:
duplicates.head()

,Title,Tags,count
0,Implementing Boundary Value Analysis of S...,c++ c,1
1,Dynamic Datagrid Binding in Silverlight?,c# silverlight data-binding,1
2,Dynamic Datagrid Binding in Silverlight?,c# silverlight data-binding columns,1
3,java.lang.NoClassDefFoundError: javax/serv...,jsp jstl,1
4,java.sql.SQLException:[Microsoft][ODBC Dri...,java jdbc,2


In [15]:
print("Number of duplicates : ", num_rows['count(*)'].values[0]-duplicates.shape[0])

Number of duplicates :  1848011


In [16]:
duplicates[count].value_counts()

NameError: name 'count' is not defined

In [3]:
if not os.path.isfile("train_w_d.db"):
    engine1 = create_engine("sqlite:///train_w_d.db")
    no_dup = pd.DataFrame(duplicates, columns=['Title', 'Body', 'Tags'])
    no_dup.to_sql('no_dup_train',engine1)

In [5]:
if os.path.isfile('train_no_dup.db'):
    con = sqlite3.connect('train_no_dup.db')
    tag_data = pd.read_sql_query("""SELECT Tags FROM no_dup_train""", con)
    print(tag_data)
    con.close()